In [1]:
import torch

from torch.utils.data import DataLoader
from torcheval.metrics import R2Score, BinaryAccuracy, BinaryAUROC

from molsetrep.utils.torch_trainer import TorchTrainer
from molsetrep.utils.datasets import molnet_loader
from molsetrep.utils.converters import molnet_to_pyg
from molsetrep.utils.root_mean_squared_error import RootMeanSquaredError
from molsetrep.utils.imbalanced_sampler import ImbalancedSampler
from molsetrep.models import SetRepClassifier, SetRepRegressor
from molsetrep.encoders import SECMQNFPEncoder, SECFPEncoder

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


# SECMQNFP

## Classification

In [4]:
train, valid, test = molnet_loader("bace_classification")
enc = SECMQNFPEncoder()

train_dataset = enc.encode(train.ids, [y[0] for y in train.y], label_dtype=torch.long)
valid_dataset = enc.encode(valid.ids, [y[0] for y in valid.y], label_dtype=torch.long)
test_dataset = enc.encode(test.ids, [y[0] for y in test.y], label_dtype=torch.long)

train_loader = DataLoader(train_dataset, batch_size=64, sampler=ImbalancedSampler(train_dataset))
valid_loader = DataLoader(valid_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
d = 44
model = SetRepClassifier(100, 16, d, 2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
criterion = torch.nn.NLLLoss()

trainer = TorchTrainer(
    model,
    optimizer,
    criterion,
    200,
    [BinaryAccuracy(), BinaryAUROC()],
    [BinaryAccuracy(), BinaryAUROC()],
    [BinaryAccuracy(), BinaryAUROC()],
    # scheduler=scheduler,
    monitor_metric=1,
    monitor_lower_is_better=False
)

trainer.train(train_loader, valid_loader)
trainer.test(test_loader)

## Regression

In [ ]:
train, valid, test = molnet_loader("lipo")
enc = SECMQNFPEncoder()

train_dataset = enc.encode(train.ids, [y[0] for y in train.y], label_dtype=torch.float)
valid_dataset = enc.encode(valid.ids, [y[0] for y in valid.y], label_dtype=torch.float)
test_dataset = enc.encode(test.ids, [y[0] for y in test.y], label_dtype=torch.float)

train_loader = DataLoader(train_dataset, batch_size=64)
valid_loader = DataLoader(valid_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
d = 44
model = SetRepRegressor(8, 32, d)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = torch.nn.MSELoss()

trainer = TorchTrainer(
    model,
    optimizer,
    criterion,
    200,
    [R2Score(), RootMeanSquaredError()],
    [R2Score(), RootMeanSquaredError()],
    [R2Score(), RootMeanSquaredError()],
    scheduler=scheduler,
    monitor_metric=1,
    monitor_lower_is_better=True
)

trainer.train(train_loader, valid_loader)
trainer.test(test_loader)

# SECFP

## Classification

In [2]:
train, valid, test = molnet_loader("bace_classification")
graph_embedder = SECFPEncoder.fit(train.ids)

enc = SECFPEncoder()

train_dataset = enc.encode(train.ids, [y[0] for y in train.y], label_dtype=torch.long, graph_embedder=graph_embedder, pretrained_graph_embedder=True)
valid_dataset = enc.encode(valid.ids, [y[0] for y in valid.y], label_dtype=torch.long, graph_embedder=graph_embedder, pretrained_graph_embedder=True)
test_dataset = enc.encode(test.ids, [y[0] for y in test.y], label_dtype=torch.long, graph_embedder=graph_embedder, pretrained_graph_embedder=True)

train_loader = DataLoader(train_dataset, batch_size=64, sampler=ImbalancedSampler(train_dataset))
valid_loader = DataLoader(valid_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
d = 500
model = SetRepClassifier(2, 16, d, 2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
criterion = torch.nn.NLLLoss()

trainer = TorchTrainer(
    model,
    optimizer,
    criterion,
    200,
    [BinaryAccuracy(), BinaryAUROC()],
    [BinaryAccuracy(), BinaryAUROC()],
    [BinaryAccuracy(), BinaryAUROC()],
    # scheduler=scheduler,
    monitor_metric=1,
    monitor_lower_is_better=False
)

trainer.train(train_loader, valid_loader)
trainer.test(test_loader)

## Regression